In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
train_dir = "./dataset/asl_alphabet_train/asl_alphabet_train"  
img_size = (128, 128) 
batch_size = 32 
chunk_size = 5
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2 
)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

class_labels = list(train_data.class_indices.keys())
print("Class Labels:", class_labels)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Class Labels: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [3]:
checkpoint_path = "asl_model.h5"

if os.path.exists(checkpoint_path):
    print("\n🔄 Loading previous checkpoint...\n")
    model = load_model(checkpoint_path)
else:
    print("\n🚀 Creating a new model...\n")
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(128, 128, 3),  
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False 

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(len(train_data.class_indices), activation="softmax")
    ])

    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])


🚀 Creating a new model...

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
checkpoint = ModelCheckpoint(
    checkpoint_path,  
    monitor="val_accuracy", 
    save_best_only=True,
    verbose=1
)

In [7]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=chunk_size,  # train for only a few epochs
    steps_per_epoch=len(train_data) // 2,
    callbacks=[checkpoint]
)

Epoch 1/5
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9810 - loss: 0.0654
Epoch 1: val_accuracy improved from -inf to 0.82310, saving model to asl_model.h5


1087/1087 ━━━━━━━━━━━━━━━━━━━━ 1726s 2s/step - accuracy: 0.9810 - loss: 0.0654 - val_accuracy: 0.8231 - val_loss: 0.7272
Epoch 2/5
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 0s 562ms/step - accuracy: 0.9857 - loss: 0.0449
Epoch 2: val_accuracy did not improve from 0.82310
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 1105s 1s/step - accuracy: 0.9857 - loss: 0.0449 - val_accuracy: 0.8097 - val_loss: 0.8216
Epoch 3/5
   1/1087 ━━━━━━━━━━━━━━━━━━━━ 6:20 350ms/step - accuracy: 0.9375 - loss: 0.0876

C:\Users\daria\.conda\envs\asl-detection-nokia\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 3: val_accuracy did not improve from 0.82310
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 798s 735ms/step - accuracy: 0.9375 - loss: 0.0876 - val_accuracy: 0.8103 - val_loss: 0.8167
Epoch 4/5
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9869 - loss: 0.0401
Epoch 4: val_accuracy improved from 0.82310 to 0.82747, saving model to asl_model.h5


1087/1087 ━━━━━━━━━━━━━━━━━━━━ 1941s 2s/step - accuracy: 0.9869 - loss: 0.0401 - val_accuracy: 0.8275 - val_loss: 0.8136
Epoch 5/5
 831/1087 ━━━━━━━━━━━━━━━━━━━━ 3:20 781ms/step - accuracy: 0.9885 - loss: 0.0354
Epoch 5: val_accuracy improved from 0.82747 to 0.82983, saving model to asl_model.h5


1087/1087 ━━━━━━━━━━━━━━━━━━━━ 1377s 1s/step - accuracy: 0.9883 - loss: 0.0358 - val_accuracy: 0.8298 - val_loss: 0.8272
